In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# requests used to get the URL
URL = "https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c"
page = requests.get(URL)
page

<Response [200]>

In [ ]:
soup = BeautifulSoup(page.content, "html.parser")

In [ ]:
# extract day and date

day_date = []
weather_day_date = soup.find_all('h3', class_ ='DailyContent--daypartName--1bzYn')           
for days in weather_day_date[1 : 21]:
  day_date.append(days.text)
print(day_date)

len(day_date)

['Sat 12 | Night', 'Sun 13 | Day', 'Sun 13 | Night', 'Mon 14 | Day', 'Mon 14 | Night', 'Tue 15 | Day', 'Tue 15 | Night', 'Wed 16 | Day', 'Wed 16 | Night', 'Thu 17 | Day', 'Thu 17 | Night', 'Fri 18 | Day', 'Fri 18 | Night', 'Sat 19 | Day', 'Sat 19 | Night', 'Sun 20 | Day', 'Sun 20 | Night', 'Mon 21 | Day', 'Mon 21 | Night', 'Tue 22 | Day']


20

In [244]:
data = []

temp_state = (data['day_date']                       # grab `description` column
                    .str.lower()                        # lower all characters for each description (row)
                    .str.contains('night')               # check for the presence of `high` within each description (True or False)
                    .map({True:'night', False:'day'})    # change `True` -> 'high' and `False` -> `low`
            )

# Adding `temp_state` to data
data['temp_state'] = temp_state

TypeError: list indices must be integers or slices, not str

In [ ]:
# extract the description
desc = []
weather_desc = soup.find_all('p', class_ ='DailyContent--narrative--hplRl')
for description in weather_desc[1 : 21]:
  desc.append(description.text)
print(desc)

len(desc)

['Showers early, then cloudy overnight. Low 49F. Winds SW at 5 to 10 mph. Chance of rain 30%.', 'Cloudy early with partial sunshine expected late. High 58F. Winds WSW at 10 to 15 mph.', 'Considerable clouds early. Some decrease in clouds late. Low near 45F. Winds W at 5 to 10 mph.', 'Mostly cloudy skies. High 61F. Winds SSW at 5 to 10 mph.', 'Overcast with rain showers at times. Low 52F. Winds SW at 5 to 10 mph. Chance of rain 60%.', 'Rain showers early with some sunshine later in the day. High near 60F. Winds WSW at 10 to 15 mph. Chance of rain 70%.', 'Partly cloudy. Low 46F. Winds WNW at 10 to 15 mph.', 'A few clouds from time to time. High 63F. Winds WNW at 10 to 20 mph.', 'Partly cloudy in the evening with more clouds for later at night. Low 47F. Winds W at 10 to 15 mph.', 'Partly to mostly cloudy. High 63F. Winds W at 10 to 15 mph.', 'Clear skies with a few passing clouds. Low 47F. Winds W at 10 to 15 mph.', 'Intervals of clouds and sunshine. High 64F. Winds W at 10 to 15 mph.', '

20

In [243]:
# extract the temperature
temp = []
weather_temp = soup.find_all('span', class_ ='DailyContent--temp--3d4dn')
for temperature in weather_temp[1 : 21]:
  temp.append(temperature.text.split("°")[0])
  temp = [int(b) for b in temp]
print(temp)

len(temp)

[49, 58, 45, 61, 52, 60, 46, 63, 47, 63, 47, 64, 48, 60, 45, 62, 45, 63, 46, 67]


20

In [ ]:
# To convert Fahrenheit to Celsius
# Formula	
# (32°F − 32) × 5/9 = 0°C
converted_temp =[]
for i in temp:
    converted_temp.append(round((i-32)*(5/9),2))        
print(converted_temp)

[9.44, 14.44, 7.22, 16.11, 11.11, 15.56, 7.78, 17.22, 8.33, 17.22, 8.33, 17.78, 8.89, 15.56, 7.22, 16.67, 7.22, 17.22, 7.78, 19.44]


In [ ]:
# join scraped data into a dataframe

weather_data = pd.DataFrame({ 'Day_Night' : day_date,
               'Temperature': converted_temp,
               'Description': desc})

weather_data

,Day_Night,Temperature,Description
0,Sat 12 | Night,9.44,"Showers early, then cloudy overnight. Low 49F...."
1,Sun 13 | Day,14.44,Cloudy early with partial sunshine expected la...
2,Sun 13 | Night,7.22,Considerable clouds early. Some decrease in cl...
3,Mon 14 | Day,16.11,Mostly cloudy skies. High 61F. Winds SSW at 5 ...
4,Mon 14 | Night,11.11,Overcast with rain showers at times. Low 52F. ...
5,Tue 15 | Day,15.56,Rain showers early with some sunshine later in...
6,Tue 15 | Night,7.78,Partly cloudy. Low 46F. Winds WNW at 10 to 15 ...
7,Wed 16 | Day,17.22,A few clouds from time to time. High 63F. Wind...
8,Wed 16 | Night,8.33,Partly cloudy in the evening with more clouds ...
9,Thu 17 | Day,17.22,Partly to mostly cloudy. High 63F. Winds W at ...
